
# Assignment 3: Gradebook & Log Analyzer

**Covers:** Conditionals & Loops • File I/O • Dictionaries, Tuples & Methods  
**Estimated time:** ~2–3 hours

## Learning outcomes
By completing this assignment, you will be able to:
1. Use conditionals and loops to implement program logic.
2. Read from and write to text/CSV files safely.
3. Model data with dictionaries and tuples; use built-in methods effectively.
4. Build a small, menu-driven CLI program that ties everything together.



## Instructions
- Work in this notebook. Replace each `# TODO` with your code.
- Do **not** change the function names or signatures unless asked.
- Run the provided tests (asserts) to self-check. Passing tests ≠ perfect score.
- When you finish, **restart & run all** to ensure it executes cleanly top-to-bottom.
- Submit the exported `.ipynb` (and any generated `.txt`/`.csv` files).

In [ ]:

# --- Setup: create tiny sample datasets (run once) ---
from pathlib import Path
import csv

data_dir = Path("data")
data_dir.mkdir(exist_ok=True)

# Sample scores.csv: student_id,name,quiz1,quiz2,quiz3
scores_path = data_dir / "scores.csv"
if not scores_path.exists():
    with scores_path.open("w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["student_id", "name", "quiz1", "quiz2", "quiz3"])
        writer.writerows([
            ["S001", "Aye", 8, 9, 10],
            ["S002", "Khin", 7, 6, 9],
            ["S003", "Ryan", 10, 10, 9],
            ["S004", "Hugo", 5, 7, 6],
            ["S005", "Thiri", 9, 8, 8],
        ])

# Sample access.log: username,action,timestamp (ISO-like, simplified)
log_path = data_dir / "access.log"
if not log_path.exists():
    log_path.write_text(
        "aye,login,2025-08-01T09:00\n"
        "aye,submit,2025-08-01T09:10\n"
        "khin,login,2025-08-01T09:05\n"
        "ryan,login,2025-08-01T09:07\n"
        "ryan,download,2025-08-01T09:20\n"
        "ryan,submit,2025-08-01T09:35\n"
        "hugo,login,2025-08-01T09:40\n"
        "thiri,login,2025-08-01T09:41\n"
        "aye,download,2025-08-01T09:50\n"
    )
print("Sample data ready at:", data_dir.resolve())



---
## Part A — Warm‑ups (Conditionals, Loops, Tuples)

### A1. `letter_grade(avg)`
Write a function that converts a numeric average (0–10) to a letter grade:
- `>= 9.0` → **A**
- `>= 8.0` → **B**
- `>= 7.0` → **C**
- `>= 6.0` → **D**
- else → **F**

Edge cases: floor to one decimal place before comparing (e.g., `8.96 → 8.9`).

### A2. `count_vowels(s)`
Return a dictionary mapping vowel → count for the string `s`. Treat `'aeiou'` as vowels and ignore case.

### A3. `rotate_tuple(t)`
Given a non-empty tuple `t`, return a **new** tuple where the last element moves to the front. Example: `(1,2,3) → (3,1,2)`.


In [10]:

# TODO: A1
def letter_grade(avg: float) -> str:
    floored = int(avg * 10) / 10  # Floor to one decimal place
    if floored >= 9.0:
        return 'A'
    elif floored >= 8.0:
        return 'B'
    elif floored >= 7.0:
        return 'C'
    elif floored >= 6.0:
        return 'D'
    else:
        return 'F'

for score in [9.0, 8.96, 8.0, 7.5, 6.0, 5.99]:
    print(f">= {score} → {letter_grade(score)}")




>= 9.0 → A
>= 8.96 → B
>= 8.0 → B
>= 7.5 → C
>= 6.0 → D
>= 5.99 → F


In [12]:

# TODO: A2
def count_vowels(s: str) -> dict:
    d ={}
    a_count = 0
    e_count = 0
    i_count = 0
    o_count = 0
    u_count = 0
    for c in s :
        if c=="A" or c=="a":
            a_count += 1
        elif c=="E" or c=="e":
            e_count += 1
        elif c=="I" or c=="i":
            i_count += 1
        elif c=="O" or c=="o":
            o_count += 1
        elif c=="U" or c=="u":
            u_count += 1
    d["A"] = a_count
    d["E"] = e_count
    d["I"] = i_count
    d["O"] = o_count
    d["U"] = u_count
    return d
print (count_vowels("Mint"))
print (count_vowels("Teacher so cool"))


{'A': 0, 'E': 0, 'I': 1, 'O': 0, 'U': 0}
{'A': 1, 'E': 2, 'I': 0, 'O': 3, 'U': 0}


In [ ]:

# TODO: A3
def rotate_tuple(t: tuple) -> tuple:
    if len(t) == 1:
        return t
    return (t[-1],) + t[:-1]
# Example tests:
print(rotate_tuple((1, 2, 3)))      
print(rotate_tuple(("a",)))         

(3, 1, 2)
('a',)


In [ ]:

# --- Tests for Part A ---
assert letter_grade(9.0) == "A"
assert letter_grade(6.0) == "D"
assert letter_grade(5.0) == "F"

assert count_vowels("Aye") == {'a': 1, 'e': 1}
assert count_vowels("Beautiful day") == {'a': 2, 'e': 1, 'i': 1, 'o': 0, 'u': 2} or        sum(count_vowels("Beautiful day").values()) == 6  # lenient check

assert rotate_tuple((1,2,3)) == (3,1,2)
assert rotate_tuple(("a",)) == ("a",)
print("Part A tests passed (if no AssertionError).")



---
## Part B — Gradebook (Files, Dicts, Methods)

You are given `data/scores.csv` with columns: `student_id,name,quiz1,quiz2,quiz3`.

### B1. `read_scores_csv(path)`
Read the CSV and return a list of dictionaries, one per student. Convert quiz scores to `int`.

### B2. `compute_averages(records)`
Given the list from B1, return a **new** list where each dict has added keys:
- `"avg"`: average of quizzes (float)
- `"grade"`: result of `letter_grade(avg)`

### B3. `write_report(records, out_path="report.txt")`
Write a simple text report with one line per student:
`S001 Aye -> avg=9.0 grade=A`

**Note:** Use `with open(...)` to handle files safely.


In [ ]:

# TODO: B1
import csv

def read_scores_csv(path: str) -> list[dict]:
    pass


In [ ]:

# TODO: B2
def compute_averages(records: list[dict]) -> list[dict]:
    pass


In [ ]:

# TODO: B3
from pathlib import Path

def write_report(records: list[dict], out_path: str = "report.txt") -> None:
    pass


In [ ]:

# --- Tests for Part B ---
recs = read_scores_csv("data/scores.csv")
assert isinstance(recs, list) and len(recs) >= 5
assert all(isinstance(r, dict) for r in recs)
assert set(recs[0].keys()) >= {"student_id","name","quiz1","quiz2","quiz3"}
assert all(isinstance(r["quiz1"], int) for r in recs)

recs2 = compute_averages(recs)
assert "avg" in recs2[0] and "grade" in recs2[0]

out_file = "report.txt"
write_report(recs2, out_file)
p = Path(out_file)
assert p.exists() and p.stat().st_size > 0
print("Part B tests passed (if no AssertionError).")



---
## Part C — Simple Log Analysis (Files, Dicts)

You are given `data/access.log` with lines: `username,action,timestamp`.

### C1. `parse_log(path)`
Return a list of tuples `(username, action, timestamp_str)`.

### C2. `action_counts(parsed)`
Return a dict {action: count} across all lines.


### C3. `most_active_user(parsed)`
Return a tuple `(username, count)` for the user with the most actions. Break ties by lexicographic username (smallest first).


In [39]:
from pathlib import Path

# TODO: C1
def parse_log(path: str) -> list[tuple]:
    result = []
    with open(path, "r") as f:
        for line in f:
            parts = line.strip().split(",")
            if len(parts) == 3:
                username, action, timestr = parts
                result.append((username, action, timestr))
    return result
# Sample access.log: username,action,timestamp (ISO-like, simplified)
data_dir = Path("data")
data_dir.mkdir(exist_ok=True)
log_path = data_dir / "access.log"

print("Sample data ready at:", data_dir.resolve())

Sample data ready at: C:\Users\mint0\Downloads\data


In [41]:
# TODO: C1
def parse_log(path: str) -> list[tuple]:
    log_path = data_dir / "access.log"
if not log_path.exists():
    log_path.write_text(
        "aye,login,2025-08-01T09:00\n"
        "aye,submit,2025-08-01T09:10\n"
        "khin,login,2025-08-01T09:05\n"
        "ryan,login,2025-08-01T09:07\n"
        "ryan,download,2025-08-01T09:20\n"
        "ryan,submit,2025-08-01T09:35\n"
        "hugo,login,2025-08-01T09:40\n"
        "thiri,login,2025-08-01T09:41\n"
        "aye,download,2025-08-01T09:50\n"
    )
print("Sample data ready at:", data_dir.resolve())


Sample data ready at: C:\Users\mint0\Downloads\data


In [ ]:

# TODO: C1
def parse_log(path: str) -> list[tuple]:
    result = []
    with open(path, "r") as f:
        for line in f:
            parts = line.strip().split(",")
            if len(parts) == 3:
                username, action, timestamp = parts
                result.append((username, action, timestamp))
    return result
#Example tests:
parsed = parse_log("data/access.log")
assert isinstance(parsed, list) and len(parsed) >= 5
assert all(isinstance(t, tuple) and len(t) == 3 for t in parsed)


In [ ]:

# TODO: C2
def action_counts(parsed: list[tuple]) -> dict:
    counts = {}
    for _, action, _ in parsed:
        counts[action] = counts.get(action, 0) + 1
    return counts
# Example tests:
sample = [("user1", "login", "time1"), ("user2", "login", "time2"), ("user1", "logout", "time3")]
result = action_counts(sample)
counts = action_counts(parsed)
assert isinstance(counts, dict) and sum(counts.values()) == len(parsed)
print(result)  # Output: {'login': 2, 'logout': 1}


{'login': 2, 'logout': 1}


In [27]:

# TODO: C3
def most_active_user(parsed: list[tuple]) -> tuple[str,int]:
    user_counts = {}
    for username, _, _ in parsed:
        user_counts[username] = user_counts.get(username, 0) + 1
    max_count = max(user_counts.values())
    # Get all users with max_count, then pick the smallest lexicographically
    top_users = [u for u, c in user_counts.items() if c == max_count]
    top_user = min(top_users)
    return (top_user, max_count)
# Example tests:
sample = [("mint","login","t1"),("mint","login","t2"),("muay","login","t3")]
user, cnt = most_active_user(sample)
print((user, cnt))  # Output should be ('mint', 2)
assert isinstance(user, str) and isinstance(cnt, int) and cnt >= 1
print("Part C tests passed (if no AssertionError).")

('mint', 2)
Part C tests passed (if no AssertionError).


In [ ]:

# --- Tests for Part C ---
parsed = parse_log("data/access.log")
assert isinstance(parsed, list) and len(parsed) >= 5
assert all(isinstance(t, tuple) and len(t) == 3 for t in parsed)

counts = action_counts(parsed)
assert isinstance(counts, dict) and sum(counts.values()) == len(parsed)

user, cnt = most_active_user(parsed)
assert isinstance(user, str) and isinstance(cnt, int) and cnt >= 1
print("Part C tests passed (if no AssertionError).")



---
## Part D — Menu‑Driven CLI (Loops, Conditionals, Methods)

Write a loop that repeatedly shows this menu until the user chooses Quit:

```
1) Show top student(s) by average
2) Show action counts from access.log
3) Export a CSV of student_id,name,avg,grade to data/gradebook_out.csv
4) Quit
```

Implement as `main()` that returns `None`. For (1), if multiple students tie for top average, print them all.


In [ ]:

# TODO: D — menu CLI
def main() -> None:
    pass

# Optional: Uncomment below to run interactively in notebook
# main()



---
## Submission checklist
- [ ] All TODOs completed
- [ ] Notebook runs top-to-bottom without errors
- [ ] `report.txt` generated
- [ ] (Optional) `data/gradebook_out.csv` generated via menu option 3